In [108]:
import glob
from  sklearn.model_selection import train_test_split
import re
import numpy as np
import os

import tensorflow as tf

import time

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Embedding, LSTM, Input
from tensorflow.keras.activations import softmax
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [109]:
files = glob.glob("data/*.txt")
X = []
Y = []

for file in files:
  lines = open(file, 'r').readlines()
  for line in lines:
    split_line = line.split("__eou__", 2)
    X.append(re.sub(r'\W+'," ", split_line[0]).strip().lower().split(" "))

    Y.append(re.sub(r'\W+'," ", split_line[1]).strip().lower().split(" "))

print(f"{X[:3]}\n{Y[:3]}")
print(f"Len X: {len(X)}")
print(f"len y: {len(Y)}")


[['trường', 'mình', 'ktx', 'sạch', 'không'], ['trường', 'mình', 'thầy', 'cô', 'nghiêm', 'khắc', 'không'], ['trường', 'mình', 'có', 'ngành', 'nào', 'liên', 'quan', 'về', 'sư', 'phạm', 'không']]
[['quá', 'sạch', 'là', 'đằng', 'khác'], ['nghiêm', 'lắm', 'ý'], ['có', 'trường', 'mình', 'có', 'khoa', 'chuyên', 'về', 'này']]
Len X: 5855
len y: 5855


In [110]:
truncated_questions = []
for question in X:
    truncated_questions.append(' '.join(question).strip())

print(truncated_questions[:3])

['trường mình ktx sạch không', 'trường mình thầy cô nghiêm khắc không', 'trường mình có ngành nào liên quan về sư phạm không']


In [111]:
truncated_answers = []
for answer in Y:
    truncated_answers.append(' '.join(answer).strip())
print(truncated_answers)

['quá sạch là đằng khác', 'nghiêm lắm ý', 'có trường mình có khoa chuyên về này', 'mình ở ktx', 'còn thiếu mình học nữa', 'trường có khoa công nghệ thong tin chuyên về máy tính nha', 'ở học kì đầu tiên thì khoảng 10 triệu đồng học kì tiếp theo cũng khoảng 10 triệu từ năm 2 trở đi thì tự chọn môn nên học phí nhiều hay ít liên quan tới đăng ký môn nhiều hay ít', 'à có khi là văn phòng khoa hoặc phòng tư vấn học đường nhưng để thuận tiện thì bạn hãy gọi điện thoại cho đỡ phải xếp hàng nhé', 'mỗi ngày trừ chủ nhật hằng tuần từ 7h30 11h30 1h 17h30', 'về việc ở ký túc xá bạn phải làm đơn khi được duyệt rồi thì sẽ đóng phí với lại phải đảm bảo vệ sinh chỗ của mình', 'thầy cô rất thân thiện với sinh viên', 'học phí tại trường mình dao động từ 10 triệu tới 20 triệu mỗi học kì', 'trường mình không có dạy ngành y', 'mình cần bằng toeic 550 để có thể tốt nghiệp ra trường', 'trường không chỉ rộng mà còn có nhiều cây xanh nữa', 'nữ sẽ mặc áo dài hồng nam mặc áo sơ mi trắng', 'o tru o ng tđt ma c đo 

In [112]:
questions_cleaned = truncated_questions
answers_cleaned = truncated_answers

remove rare word


In [113]:
# Create a word count dictionary for question and answer vocabulary to find the rarely occuring words
count_mapping_dict = dict()
def data_imporvement(text):
    for word in text.split():
        if word not in count_mapping_dict:
            count_mapping_dict[word] = 1
        else:
            count_mapping_dict[word] += 1

In [114]:
# find the question vocab counts
for text in questions_cleaned:
    data_imporvement(text)

In [115]:
# find the answers vocab counts
for text in answers_cleaned:
    data_imporvement(text)

In [116]:
# Creating threshold to filter out less frequent words
# Filter out words from vocabulary whose count is less than the threshold
# create a words to int dictionary for question words
threshold = 40
questionwords2int ={}
word_number = 0
for word, count in count_mapping_dict.items():
         if count > threshold:
            questionwords2int[word] =  word_number
            word_number += 1

In [117]:
# Verify the length of mapping dict or the number of words in the question vocabulary
len(questionwords2int)

372

In [118]:

# Creating threshold to filter out less frequent words
# Filter out words from vocabulary whose count is less than the threshold
# create a words to int dictionary for answer words
threshold = 40
answerwords2int = {}
word_number = 0
for word, count in count_mapping_dict.items():
         if count > threshold:
            answerwords2int[word] =  word_number
            word_number += 1

In [119]:
len(answerwords2int)

372

In [120]:
# <SOS> Start of string
# <EOS> End of String
# <PAD> for maintaining the length of input
# <OUT> for words not used while training(filter out)
tokens = ['<PAD>', '<EOS>', '<OUT>', '<SOS>']

# adding token and corresponding integer mapping to existing word to integer mapping for questions
for token in tokens:
    questionwords2int[token] = len(questionwords2int)

In [121]:

# adding token and corresponding integer mapping to existing word to integer mapping for answers
for token in tokens:
    answerwords2int[token] = len(answerwords2int)

In [122]:
# create an inverse dictionary of answers2int for decoder so that after predicting the owrd index we can create the words
int2answerwords = {i:w for w,i in answerwords2int.items()}
print(int2answerwords)

{0: 'trường', 1: 'mình', 2: 'không', 3: 'cô', 4: 'có', 5: 'ngành', 6: 'nào', 7: 'quan', 8: 'về', 9: 'bạn', 10: 'ở', 11: 'nhà', 12: 'trọ', 13: 'hay', 14: 'thấy', 15: 'còn', 16: 'gì', 17: 'khoa', 18: 'những', 19: 'máy', 20: 'tính', 21: 'vậy', 22: 'cho', 23: 'hỏi', 24: 'học', 25: 'phí', 26: 'của', 27: 'từng', 28: 'với', 29: 'ạ', 30: 'khi', 31: 'cần', 32: 'phải', 33: 'thì', 34: 'sẽ', 35: 'đến', 36: 'phòng', 37: 'để', 38: 'giờ', 39: 'động', 40: 'vào', 41: 'khoảng', 42: 'thời', 43: 'gian', 44: 'ơi', 45: 'muốn', 46: 'yêu', 47: 'cầu', 48: 'trong', 49: 'thân', 50: 'sinh', 51: 'viên', 52: 'tại', 53: 'y', 54: 'bằng', 55: 'bao', 56: 'nhiêu', 57: 'ra', 58: 'mặc', 59: 'đồng', 60: 'như', 61: 'thế', 62: 'o', 63: 'ng', 64: 'c', 65: 'thứ', 66: 'mấy', 67: 'cơ', 68: 'sở', 69: 'chất', 70: 'nghe', 71: 'nói', 72: 'rất', 73: 'i', 74: 'cao', 75: 'u', 76: 'mi', 77: 'nh', 78: 'co', 79: 'gia', 80: 'n', 81: 'em', 82: 'kinh', 83: 'a', 84: 't', 85: 'tôi', 86: 'đâu', 87: 'được', 88: 'tôn', 89: 'đức', 90: 'thắng', 91:

addd tag

In [123]:
for i in range(len(answers_cleaned)):
    answers_cleaned[i] = '<SOS> '+ answers_cleaned[i] + ' <EOS>'
print(answers_cleaned[:3])

['<SOS> quá sạch là đằng khác <EOS>', '<SOS> nghiêm lắm ý <EOS>', '<SOS> có trường mình có khoa chuyên về này <EOS>']


translate

In [124]:
# Converting question setences to integer encoding
question_to_int = []
for question in questions_cleaned:
    ints = []
    for word in question.split():
        if word not in questionwords2int:
            ints.append(questionwords2int['<OUT>'])
        else:
            ints.append(questionwords2int[word])
    question_to_int.append(ints)

In [125]:
# Converting answer setences to integer encoding
answer_to_int = []
for answer in answers_cleaned:
    ints = []
    for word in answer.split():
        if word not in answerwords2int:
            ints.append(answerwords2int['<OUT>'])
        else:
            ints.append(answerwords2int[word])
    answer_to_int.append(ints)

In [126]:
# Calculate the length of longest question to be used for encode model input
question_sequence_length = max([len(question) for question in question_to_int])
question_sequence_length

93

In [127]:
# Calculate the length of longest answer to be used for decoder model input
answer_sequence_length = max([len(answer) for answer in answer_to_int])
answer_sequence_length

158

In [128]:
# Create the LSTM based Encoder-Decoder model using Keras Functional API
# dimension of embedding layer
EMBED_HID_DIM = 100

# dimension of LSTM unit
latent_dim = 100

# Size of questions and answers vocab
vocab_questions = len(questionwords2int)
vocab_answers = len(answerwords2int)

# Encoder Model creationS
# Define an input shape.
encoder_inputs = Input(shape=(None,))

# Define the embedding layer
inputs_embed = Embedding(input_dim=vocab_questions, output_dim=EMBED_HID_DIM, input_length=question_sequence_length)
encoder_embed = inputs_embed(encoder_inputs)

# DEFINE THE LSTM layer
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_embed)

# create the encoder model
model = Model(encoder_inputs, encoder_outputs)


# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.

#decoder input shape
decoder_inputs = Input(shape=(None,))

# define decoder embedding layer
decode_inputs_embed = Embedding(input_dim=vocab_answers, output_dim=EMBED_HID_DIM, input_length=answer_sequence_length)
decoder_embed = decode_inputs_embed(decoder_inputs)

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the 
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embed,
                                     initial_state=encoder_states)

# Final classifier layer Dense and softmax activated
decoder_dense = Dense(vocab_answers, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.summary()

Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, None, 100)    37600       input_7[0][0]                    
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, None, 100)    37600       input_8[0][0]                    
____________________________________________________________________________________________

In [129]:
# Compile the Keras model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics='acc')

data preparetion


In [130]:
# Padding of Integer coded question and answers with <PAD> sequence
def padding(encoder_sequences, decoder_sequences):
    
    encoder_input_data = pad_sequences(encoder_sequences, maxlen=question_sequence_length, dtype='int32', padding='post', truncating='post', value= questionwords2int['<PAD>'])
    decoder_input_data = pad_sequences(decoder_sequences, maxlen=answer_sequence_length, dtype='int32', padding='post', truncating='post', value= answerwords2int['<PAD>'])
  
    return encoder_input_data, decoder_input_data

encoder_input_data, decoder_input_data = padding(question_to_int, answer_to_int)

In [131]:
# Verify encoder input shape
encoder_input_data.shape

(5855, 93)

In [132]:
# Verify encoder input shape
decoder_input_data.shape

(5855, 158)

In [133]:
# Function to generate batch of training data based on batch size becuase dataset is too big to fit in memory
# encoder_input_data = as is it
# decoder_input data = as it is
# decoder_target_data = offset by one timestep

max_source_length = question_sequence_length
max_target_length = answer_sequence_length
num_decoder_tokens = vocab_answers

def generate_batch(X , y , batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):

            encoder_input_data = np.zeros((batch_size, max_source_length),dtype='int32')
            decoder_input_data = np.zeros((batch_size, max_target_length),dtype='int32')
            decoder_target_data = np.zeros((batch_size, max_target_length, num_decoder_tokens),dtype='int32')

            for i, (input_seq, target_seq) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_seq):
                    encoder_input_data[i, t] = word
                for t, word in enumerate(target_seq):
                    if t<len(target_seq)-1:
                        decoder_input_data[i, t] = word # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        #print(word)
                        decoder_target_data[i, t - 1, word] = 1.


            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [158]:
# Model training configuration
train_samples = len(question_to_int) # Total Training samples
batch_size = 128
epochs = 25

In [159]:
X_train = encoder_input_data
y_train = decoder_input_data

In [160]:
# Model fit
model.fit(generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size, epochs=epochs)

Epoch 1/25
45/45 [==============================] - 39s 855ms/step - loss: 0.2756 - acc: 0.9466
Epoch 2/25
45/45 [==============================] - 38s 846ms/step - loss: 0.2741 - acc: 0.9400
Epoch 3/25
45/45 [==============================] - 38s 845ms/step - loss: 0.2674 - acc: 0.9421
Epoch 4/25
45/45 [==============================] - 38s 842ms/step - loss: 0.2649 - acc: 0.9421
Epoch 5/25
45/45 [==============================] - 38s 845ms/step - loss: 0.2643 - acc: 0.9420
Epoch 6/25
45/45 [==============================] - 38s 843ms/step - loss: 0.2622 - acc: 0.9422
Epoch 7/25
45/45 [==============================] - 38s 846ms/step - loss: 0.2593 - acc: 0.9425
Epoch 8/25
45/45 [==============================] - 38s 851ms/step - loss: 0.2614 - acc: 0.9421
Epoch 9/25
45/45 [==============================] - 43s 947ms/step - loss: 0.2612 - acc: 0.9421
Epoch 10/25
45/45 [==============================] - 43s 951ms/step - loss: 0.2588 - acc: 0.9424
Epoch 11/25
45/45 [====================

In [161]:
# Save the model to disk
model.save('model/LSTM_chatbot.h5')

In [162]:
# Encode the input sequence to get the "Context vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup

# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_state_input = [decoder_state_input_h, decoder_state_input_c]

# Get the embeddings of the decoder sequence
dec_emb2 = decode_inputs_embed(decoder_inputs)

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_state_input)
decoder_states2 = [state_h2, state_c2]

# A dense softmax layer to generate prob dist. over the target vocabulary
decoder_outputs2 = decoder_dense(decoder_outputs2)

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_state_input,
    [decoder_outputs2] + decoder_states2)

In [186]:
# Function to decode the sequence from a decoder given the input sequence
# input_seq = input()
def decode_sequence(input_seq):
        # Encode the input as state vectors.
        states_value = encoder_model.predict(input_seq)
        
        # Generate empty target sequence of length 1.
        target_seq = np.zeros((1,1))
        
        # Populate the first character of 
        #target sequence with the start character.
        target_seq[0, 0] = answerwords2int['<SOS>']
        
        # Sampling loop for a batch of sequences
        # (to simplify, here we assume a batch of size 1).
        stop_condition = False
        decoded_sentence = ' '
        
        while not stop_condition:
            output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
            
            # Sample a token
            sampled_token_index = np.argmax(output_tokens[0, -1, :])
            sampled_word =int2answerwords[sampled_token_index]
            decoded_sentence += ' '+ sampled_word
            
            # Exit condition: either hit max length
            # or find stop character.
            if (sampled_word == '<EOS>' or len(decoded_sentence.split()) > 25):
                stop_condition = True
        
            # Update the target sequence (of length 1).
            target_seq = np.zeros((1,1))
            target_seq[0, 0] = sampled_token_index
            
            # Update states
            states_value = [h, c]
        return decoded_sentence
# decode_sequence(input_seq)
# print(states_value)

bạn là ai


IndexError: ignored

In [181]:
# Create a batch generator for batch size 1
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=2

In [192]:
while (True):
  input_seq = input("You: ")
  if (input_seq == "exit"):
    break
  for i in range(0,len(questions_cleaned)-1) :
    if(input_seq == questions_cleaned[i]):
      decoded_sentence = decode_sequence(input_seq)
      print("Ans: ",decoded_sentence)
      
    else:
      print("Ans: ")
      
len(questions_cleaned)     

You: bạn là ai
Ans:   quá sạch là đằng khác 
Ans:   nghiêm lắm ý 
Ans:   có trường mình có khoa chuyên về này 
Ans:   mình ở ktx 
Ans:   còn thiếu mình học nữa 
Ans:   trường có khoa công nghệ thong tin chuyên về máy tính nha 
Ans:   ở học kì đầu tiên thì khoảng 10 triệu đồng học kì tiếp theo cũng khoảng 10 triệu từ năm 2 trở đi thì tự chọn môn nên học phí nhiều hay ít liên quan tới đăng ký môn nhiều hay ít 
Ans:   à có khi là văn phòng khoa hoặc phòng tư vấn học đường nhưng để thuận tiện thì bạn hãy gọi điện thoại cho đỡ phải xếp hàng nhé 
Ans:   mỗi ngày trừ chủ nhật hằng tuần từ 7h30 11h30 1h 17h30 
Ans:   về việc ở ký túc xá bạn phải làm đơn khi được duyệt rồi thì sẽ đóng phí với lại phải đảm bảo vệ sinh chỗ của mình 
Ans:   thầy cô rất thân thiện với sinh viên 
Ans:   học phí tại trường mình dao động từ 10 triệu tới 20 triệu mỗi học kì 
Ans:   trường mình không có dạy ngành y 
Ans:   mình cần bằng toeic 550 để có thể tốt nghiệp ra trường 
Ans:   trường không chỉ rộng mà còn có nhi

IndexError: ignored

In [190]:
# Predict the target sentence and compare with the actual target sentence given a source sentence
k+=1
(input_seq, actual_output), target_output = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Source sentence:',questions_cleaned[k:k+1][0])
print('Actual Target Sentence:', answers_cleaned[k:k+1][0][5:-5])
print('Predicted Target Sentence:', decoded_sentence)

Input Source sentence: bạn thấy trường mình còn thiếu gì không
Actual Target Sentence:  còn thiếu mình học nữa 
Predicted Target Sentence:   mình <OUT> <OUT> <OUT> <OUT> <OUT> <OUT> <OUT> <OUT> <OUT> <OUT> <OUT> <OUT> <OUT> <OUT> <OUT> <OUT> <OUT> <OUT> <OUT> <OUT> <OUT> <OUT> <OUT> <OUT> <OUT>
